# Regresión con máquinas de soporte vectorial
 en el Problema de Viviendas de Boston



In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import QuantileRegressor

from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [33]:
df = pd.read_csv('../data/BostonHousing.csv')
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


## Regresión OLS vs SVR

### OLS

In [34]:
X = df[['DIS', 'RM', 'AGE']].values
y = df['MEDV'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [35]:
lr = LinearRegression().fit(X_train, y_train)
lr_pred = lr.predict(X_test)

### SVR 
Hacemos una regresión con máquinas de soporte vectorial, ventajas:

*	Relaciones claramente no lineales o con interacciones complejas.
*	Datos con outliers (no extremos) donde OLS sufre por la pérdida cuadrática.
*	Muchas features y pocas muestras (alta dimensión): los márgenes grandes tienden a generalizar mejor que OLS.

Desventajas:

*	Requiere tuning cuidadoso (C, epsilon, kernel, gamma).
*	SVR con kernel no escala bien a datasets muy grandes (miles–decenas de miles de muestras); en esos casos considerar LinearSVR o métodos de boosting/GBDT.
*	Sensible al escalado; sin escalar, el rendimiento cae.
*	No ofrece intervalos/probabilidades directamente (a diferencia de GLMs).

In [36]:
pipe = make_pipeline(StandardScaler(), SVR())
param_grid = {
    "svr__kernel": ["rbf"],
    "svr__C": [0.1, 1, 10, 100],
    "svr__epsilon": [0.01, 0.1, 0.5],
    "svr__gamma": ["scale", "auto"]
}
search = GridSearchCV(pipe, param_grid, cv=5, n_jobs=-1)
svr_model = search.fit(X_train, y_train)
svr_pred = svr_model.predict(X_test)

## Comparativa

In [38]:
print("Regresión OLS — R^2 de prueba:", round(r2_score(y_test, lr_pred), 3))
print("Regresión SVR — R^2 de prueba:", round(r2_score(y_test, svr_pred), 3))
print("-----------------------------------------")
print("Regresión OLS — MAE de prueba:", round(mean_absolute_error(y_test, lr_pred), 3))
print("Regresión SVR — MAE de prueba:", round(mean_absolute_error(y_test, svr_pred), 3))


Regresión OLS — R^2 de prueba: 0.476
Regresión SVR — R^2 de prueba: 0.638
-----------------------------------------
Regresión OLS — MAE de prueba: 3.898
Regresión SVR — MAE de prueba: 3.057
